### TODO
- Everything
- Define the best approach to the collaborative filtering (model-based or memory-based)
- Build the recommender system (can it be as simple as kNN?)

### DONE
- File loading and creation of the DataFrame

In [29]:
# Imports
import pandas as pd
import numpy as np

In [30]:
# Read Movie Titles file
def readMovieTitle(file_path):
    data_dict = {'Movie_Id' : [], 'Release_Year' : [], 'Title' : []}
    data_file = open(file_path, "r", encoding='ISO-8859-1')
    for line in data_file:            
        id, year, title = line.split(',', 2)
        data_dict['Movie_Id'].append(id)
        data_dict['Release_Year'].append(year)
        data_dict['Title'].append(title.rstrip('\n'))
    data_file.close()
            
    return pd.DataFrame(data_dict)

In [31]:
# The dataset is very large, so there's a flag to load only a handful of rows if necessary
# First 100k lines takes 0.2s to load, the whole dataset takes almost 2 minutes
def readFile(file_path, rows = 100000, flag = False):
    data_dict = {'Cust_Id' : [], 'Movie_Id' : [], 'Rating' : [], 'Date' : []}
    data_file = open(file_path, "r")
    count = 0
    for line in data_file:
        count += 1
        if flag and (count > rows):
            break
            
        if ':' in line:
            movidId = line[:-2] # remove the last character ':'
            movieId = int(movidId)
        else:
            customerID, rating, date = line.split(',')
            data_dict['Cust_Id'].append(customerID)
            data_dict['Movie_Id'].append(movieId)
            data_dict['Rating'].append(rating)
            data_dict['Date'].append(date.rstrip("\n"))
    data_file.close()
            
    return pd.DataFrame(data_dict)

In [32]:
# Load the title of the movies
df_title = readMovieTitle('data/movie_titles.csv')
df_title['Movie_Id'] = df_title['Movie_Id'].astype(int)
df_title.head(10)

,Movie_Id,Release_Year,Title
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW
5,6,1997,Sick
6,7,1992,8 Man
7,8,2004,What the #$*! Do We Know!?
8,9,1991,Class of Nuke 'Em High 2
9,10,2001,Fighter


In [33]:
df_title.loc[df_title['Movie_Id'] == '4500']

,Movie_Id,Release_Year,Title


In [34]:
# Loading the user data
# User ID, Movie ID, Rating and Date
flag_limit = True
df1 = readFile('data/combined_data_1.txt', flag = flag_limit)
df2 = readFile('data/combined_data_2.txt', flag = flag_limit)
df3 = readFile('data/combined_data_3.txt', flag = flag_limit)
df4 = readFile('data/combined_data_4.txt', flag = flag_limit)
df1['Rating'] = df1['Rating'].astype(float)
df2['Rating'] = df2['Rating'].astype(float)
df3['Rating'] = df3['Rating'].astype(float)
df4['Rating'] = df4['Rating'].astype(float)

In [35]:
# Creation of the DataFrame with all the data
df = df1.copy()
df = pd.concat([df2, df3, df4])
df.index = np.arange(0,len(df))
df.head(10)

# Complete Dataframe with the movie titles
# Not necessary
df = df.merge(df_title, how='left')
df = df.loc[:, ['Cust_Id', 'Movie_Id', 'Title', 'Release_Year', 'Rating', 'Date']]
df.head(-10)

# Too much data, so I'm deleting the dataframes after the merge
# In total it uses 14gb of memory
del df1, df2, df3, df4

,Cust_Id,Movie_Id,Title,Release_Year,Rating,Date
0,2532865,4500,Les Dames du Bois de Boulogne,1945,4.0,2005-07-26
1,573364,4500,Les Dames du Bois de Boulogne,1945,3.0,2005-06-20
2,1696725,4500,Les Dames du Bois de Boulogne,1945,3.0,2004-02-27
3,1253431,4500,Les Dames du Bois de Boulogne,1945,3.0,2004-03-31
4,1265574,4500,Les Dames du Bois de Boulogne,1945,2.0,2003-09-01
...,...,...,...,...,...,...
76426728,925414,17770,Alien Hunter,2003,4.0,2005-02-08
76426729,1983962,17770,Alien Hunter,2003,2.0,2005-02-23
76426730,897629,17770,Alien Hunter,2003,3.0,2005-04-13
76426731,1274035,17770,Alien Hunter,2003,4.0,2005-06-10
